In [1]:
import pandas as pd
import numpy as np
import re
import pyodbc
from sqlalchemy import create_engine
import urllib
import datetime as dt
from pathlib3x import Path
import easygui

In [2]:

def selecionarArquivo(*pasta):
    """
    Selecionar um arquivo para servir de carga no SAP
    Returns:
        String: retorna um caminho em forma de string (texto)
    """
    global local_arquivo
    local_arquivo = ''

    if not pasta:
        arquivo = easygui.fileopenbox()
        local_arquivo = Path(arquivo)   
    else:
        print(f"o arquivo da pasta: {pasta[0]}")
        local_arquivo = pasta[0]
        
    return local_arquivo

###### Selecionar a pasta do arquivo do mês ########

localArquivo = selecionarArquivo()

In [3]:
localArquivo

WindowsPath('W:/04 - Diretoria Financeira/Contas-a-pagar/NESA/Financeiro/ANO 2023/APLICAÇÕES FINANCEIRAS/Carga_SQL/SQL_base_Aplicacao_04-10-2023.xlsx')

In [4]:
"""
Banco de dados: 

[DW_SFCRI].[TESOURARIA].[APLICACAO_FUNDOS]
[DW_SFCRI].[TESOURARIA].[TITULOS]
[DW_SFCRI].[TESOURARIA].[BANCOS]


sheets: 

TITULOS, APLICACAO_FUNDOS, BANCOS

"""


'\nBanco de dados: \n\n[DW_SFCRI].[TESOURARIA].[APLICACAO_FUNDOS]\n[DW_SFCRI].[TESOURARIA].[TITULOS]\n[DW_SFCRI].[TESOURARIA].[BANCOS]\n\n\nsheets: \n\nTITULOS, APLICACAO_FUNDOS, BANCOS\n\n'

In [4]:
df_bancos = pd.read_excel(localArquivo, sheet_name='BANCOS')
df_titulos = pd.read_excel(localArquivo, sheet_name='TITULOS')
df_aplicacao = pd.read_excel(localArquivo, sheet_name='APLICACAO_FUNDOS')

In [5]:
df_titulos['LIQUIDEZ'] = df_titulos['LIQUIDEZ'].astype(str)
df_titulos

,TIPO,EMISSOR,APLICACAO,NUM_SAP_TRM,DATA_APLICACAO,DATA_VENCIMENTO,LIQUIDEZ,CP_LP,TIPO_APLICACAO,DIAS_PARA_LIQUIDEZ,TAXA_CONTRATADA
0,FUNDOS,CEF FUNDO XINGU,CEF_FXINGU_BNDES,100000000042,NaT,2042-01-31,Diária,NaN,Fundo,0,1.13
1,FUNDOS,CEF FUNDO XINGU,CEF_FXINGU_AGENTES,100000000043,NaT,2042-01-31,Diária,NaN,Fundo,0,1.13
2,FUNDOS,CEF FUNDO XINGU,CEF_FXINGU_OM,100000000045,NaT,2042-01-31,Diária,NaN,Fundo,0,1.13
3,FUNDOS,CEF FUNDO XINGU,CEF_FXINGU_ONS,100000000048,NaT,2042-01-31,Diária,NaN,Fundo,0,1.13
4,FUNDOS,CEF FUNDO XINGU,CEF_FXINGU_IBAMA,100000000046,NaT,2042-01-31,Diária,NaN,Fundo,0,1.13
...,...,...,...,...,...,...,...,...,...,...,...
203,APLICACOES,ITAU,ITAU_LF_2605_01,LFSC190006N,2021-05-26,2029-01-08,Diária,LP,LF,0,1.35
204,APLICACOES,ITAU,ITAU_LF_2605_02,LFSC1900072,2021-05-26,2029-01-08,Diária,LP,LF,0,1.35
205,APLICACOES,ITAU,ITAU_LF_2605_03,LFSC190006P,2021-05-26,2029-07-16,Diária,LP,LF,0,1.35
206,APLICACOES,ITAU,ITAU_LF_2605_04,LFSC190006S,2021-05-26,2029-07-16,Diária,LP,LF,0,1.35


In [6]:
df_bancos

,CODIGO,NATUREZA_BANCO,BANCO,GRUPO_POLITICA,DATA_PL,S&P,FITCH,MOODYS,INDICE_BASILEIA,PL_RS_MIL,PERC_EXPOSICAO_POLITICA,LIMITE _BANCO
0,DFs BACEN,Público,BACEN,Grupo 1,2022-12-31,NaN,NaN,NaN,0.0000,61012000000,0.08,4.880960e+09
1,1000080329,Público,BB,Grupo 1,2022-12-31,NaN,NaN,NaN,0.1619,159259546000,0.08,1.274076e+10
2,1000080738,Público,CEF,Grupo 1,2022-12-31,AAA,AA,AA,0.1763,87146100000,0.08,6.971688e+09
3,1000081593,Público,BNB,Grupo 1,2022-12-31,NaN,NaN,NaN,0.1337,9496301000,0.08,7.597041e+08
4,1000080288,Público,BRB,Grupo 1,2022-12-31,NaN,AA,NaN,0.1491,2630660000,0.08,2.104528e+08
5,1000080099,Privado,ITAU,Grupo 2,2022-12-31,AAA,AAA,A,0.1498,175028066000,0.07,1.225196e+10
6,1000080075,Privado,BRADESCO,Grupo 2,2022-12-31,NaN,AAA,NaN,0.1509,156238051000,0.07,1.093666e+10
7,1000080336,Privado,BTG,Grupo 2,2022-12-31,AA,AA,AA,0.1552,48540974000,0.06,2.912458e+09
8,1000080109,Privado,SAFRA,Grupo 2,2022-12-31,AAA,NaN,NaN,0.1841,23898959000,0.07,1.672927e+09
9,1000082475,Privado,XP,Grupo 3,2022-12-31,NaN,AA,NaN,0.2035,9536003000,0.05,4.768002e+08


In [7]:
df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].astype(str)
df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].str.strip()
# Suponha que 'df_aplicacao' seja o seu DataFrame e 'VALOR_APLICACAO' seja a coluna que você deseja converter
df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].apply(lambda x: float(x) if x.strip() else 0.0)

In [8]:
df_aplicacao

,TIPO,APLICACAO,DATA,CDI_100%,FATOR_DIARIO,VALOR_APLICACAO,RESGATE,VALOR_COTA,QTD_COTAS,GANHO_DIARIO,SALDO_BRUTO
0,FUNDOS,CEF_FXINGU_BNDES,2020-10-31,1.000075,1.000084,NaN,NaN,2.037246,2.422558e+07,0,4.935346e+07
1,FUNDOS,CEF_FXINGU_BNDES,2020-11-03,1.000075,1.000084,NaN,NaN,2.037412,2.422558e+07,4026.776788,4.935748e+07
2,FUNDOS,CEF_FXINGU_BNDES,2020-11-04,1.000075,1.000084,NaN,NaN,2.037537,2.422558e+07,3038.371864,4.936052e+07
3,FUNDOS,CEF_FXINGU_BNDES,2020-11-05,1.000075,1.000084,NaN,NaN,2.037558,2.422558e+07,490.567934,4.936101e+07
4,FUNDOS,CEF_FXINGU_BNDES,2020-11-06,1.000075,1.000084,NaN,NaN,2.037627,2.422558e+07,1681.497298,4.936269e+07
...,...,...,...,...,...,...,...,...,...,...,...
125594,APLICACOES,ITAU_290923_80,2023-10-23,1.000473,1.000378,NaN,NaN,1.005688,5.249921e+05,199.621796,5.279785e+05
125595,APLICACOES,ITAU_290923_80,2023-10-24,1.000473,1.000378,NaN,NaN,1.006069,5.249921e+05,199.697299,5.281782e+05
125596,APLICACOES,ITAU_290923_80,2023-10-25,1.000473,1.000378,NaN,NaN,1.006449,5.249921e+05,199.77283,5.283779e+05
125597,APLICACOES,ITAU_290923_80,2023-10-26,1.000473,1.000378,NaN,NaN,1.006830,5.249921e+05,199.84839,5.285778e+05


#### carga SQL

In [9]:
conexao = pyodbc.connect('Driver={SQL Server};'
                      'Server=CURIAIA1-10-3\FINANCEIRO_PRD;'
                      'Database=DW_SFCRI;'
                      'Trusted_Connection=yes;')

cursor = conexao.cursor() # Cria o cursor para manipular o banco de dados
truncate_APLICACAO_FUNDOS = """ TRUNCATE TABLE [DW_SFCRI].[TESOURARIA].[APLICACAO_FUNDOS] """
# truncate_APLICACAO_FUNDOS = """ TRUNCATE TABLE [DW_SFCRI].[TESOURARIA].[APLICACAO_FUNDOS_2] """
truncate_BANCOS = """ TRUNCATE TABLE [DW_SFCRI].[TESOURARIA].[BANCOS] """
truncate_TITULOS = """ TRUNCATE TABLE [DW_SFCRI].[TESOURARIA].[TITULOS] """

In [10]:
cursor.execute(truncate_APLICACAO_FUNDOS) # Executa a Limpeza dos dados 
cursor.execute(truncate_BANCOS) # Executa a Limpeza dos dados 
cursor.execute(truncate_TITULOS) # Executa a Limpeza dos dados 
cursor.commit() # Confirma

In [11]:
pst = urllib.parse.quote_plus('Driver={SQL Server};'
                                'Server=CURIAIA1-10-3\FINANCEIRO_PRD;'
                                'Database=DW_SFCRI;'
                                'Trusted_Connection=yes;')
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={pst}')

In [12]:
df_titulos.to_sql('TITULOS', con=engine, if_exists='append', index=False, schema="TESOURARIA")
df_bancos.to_sql('BANCOS', con=engine, if_exists='append', index=False, schema="TESOURARIA")
print('Começando a carga de aplicação')
# df_aplicacao.to_sql('APLICACAO_FUNDOS_2', con=engine, if_exists='replace', index=False, schema="TESOURARIA")
df_aplicacao.to_sql('APLICACAO_FUNDOS', con=engine, if_exists='replace', index=False, schema="TESOURARIA")
print('Fim da carga de aplicação')


Começando a carga de aplicação
Fim da carga de aplicação


#### Testes tipos

In [14]:
# Suponha que 'df_aplicacao' seja o seu DataFrame e 'VALOR_APLICACAO' seja a coluna que deseja limpar

# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].astype(str)
# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].str.strip()

In [15]:
# Suponha que 'df_aplicacao' seja o seu DataFrame e 'VALOR_APLICACAO' seja a coluna que você deseja converter
# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].apply(lambda x: float(x) if x.strip() else 0.0)


In [16]:

# Suponha que 'df_aplicacao' seja o seu DataFrame e 'VALOR_APLICACAO' seja a coluna que você deseja converter
# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].apply(lambda x: float(x.strip()) if isinstance(x, str) and x.strip() else 0.0)


In [17]:
# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].astype(float)